## Visualize training runs

When you are building machine learning models with TensorFlow, we'll be spending a lot of time training the model and then repeating the training process with different parameters to see what works best. Using TensorBoard we can visually monitor the progress of training as it happens, and even compare different training runs against each other.



we've defined the training filewriter and a testing filewriter that will write out log files during training that we can then view in TensorBoard. Let's say that we want to retrain this neural network several times, with a different number of nodes in the first layer each time.

**Our goal** is to find out which neural network design gives us the best prediction accuracy. The problem is that each time we run the training process, additional log files will be created with the same name as the old log files. The new log files will mix with the old log files and create overlapping graphs in TensorBoard. We won't have any way to tell which training run was which.

 We can fix this by using a different run name for each training run.
 
Let's create the variable called RUN_NAME. This will just be a string where we can give a name to the current training run. Let's start with "run 1 with 50 nodes". 


Currently every log file is named exactly the same way. Let's change how we are creating the log file names, so that we can separate them per run. All we have to do is put each run in a different subfolder. To do that, let's add in run name to the path where we are saving the log files. So here I'll add slash and then the placeholder, and then I'll substitute that in by calling .format and passing in the run name. And we'll do the same thing for the testing writer. .format, and pass in the run name. 

In [8]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

tf.reset_default_graph() 
#In the case you want to re-run codeblock 3 (for what ever reason) just insert a simple tf.reset_default_graph()
#at the beginning of the block. 
#This will reset the graph you have already create and though you can create it again.


# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

# Define model parameters
##########################################################
# Adding RUN_NAME
#########################################################
RUN_NAME = "run 1 with 50 nodes"
learning_rate = 0.001
training_epochs = 100

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Section One: Define the layers of the neural network itself

# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="X")

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# Section Two: Define the cost function of the neural network that will be optimized during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    
###############################################################
# Saving Different log file names
##############################################################
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every few training steps, log our progress
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))


Epoch: 0 - Training Cost: 0.0998048335313797  Testing Cost: 0.10595479607582092
Epoch: 5 - Training Cost: 0.031163925305008888  Testing Cost: 0.032601162791252136
Epoch: 10 - Training Cost: 0.022592948749661446  Testing Cost: 0.023315947502851486
Epoch: 15 - Training Cost: 0.010349605232477188  Testing Cost: 0.011542453430593014
Epoch: 20 - Training Cost: 0.009305378422141075  Testing Cost: 0.010597547516226768
Epoch: 25 - Training Cost: 0.0045861657708883286  Testing Cost: 0.005110942758619785
Epoch: 30 - Training Cost: 0.004168755374848843  Testing Cost: 0.004293614998459816
Epoch: 35 - Training Cost: 0.002657415345311165  Testing Cost: 0.0028235369827598333
Epoch: 40 - Training Cost: 0.00260140816681087  Testing Cost: 0.0028646260034292936
Epoch: 45 - Training Cost: 0.001934573519974947  Testing Cost: 0.002011549659073353
Epoch: 50 - Training Cost: 0.001592321670614183  Testing Cost: 0.0016639715759083629
Epoch: 55 - Training Cost: 0.0013818895677104592  Testing Cost: 0.001472899923

Let's try tweaking our neural network and doing a second run.

First, let's redefined our  RUN_Namre="run 2 with 20 nodes".
And then let's edit the number of nodes in the first layer to be 20.

In [9]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

tf.reset_default_graph() 
#In the case you want to re-run codeblock 3 (for what ever reason) just insert a simple tf.reset_default_graph()
#at the beginning of the block. 
#This will reset the graph you have already create and though you can create it again.


# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

# Define model parameters
##########################################################
# Adding RUN_NAME
#########################################################
RUN_NAME = "run 2 with 20 nodes"
learning_rate = 0.001
training_epochs = 100

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 20
layer_2_nodes = 100
layer_3_nodes = 50

# Section One: Define the layers of the neural network itself

# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="X")

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# Section Two: Define the cost function of the neural network that will be optimized during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    
###############################################################
# Saving Different log file names
##############################################################
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every few training steps, log our progress
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))


Epoch: 0 - Training Cost: 0.11850441247224808  Testing Cost: 0.12488681823015213
Epoch: 5 - Training Cost: 0.022354397922754288  Testing Cost: 0.024478714913129807
Epoch: 10 - Training Cost: 0.033058442175388336  Testing Cost: 0.033243052661418915
Epoch: 15 - Training Cost: 0.019600527361035347  Testing Cost: 0.020515475422143936
Epoch: 20 - Training Cost: 0.015040948987007141  Testing Cost: 0.016889413818717003
Epoch: 25 - Training Cost: 0.015711022540926933  Testing Cost: 0.017822954803705215
Epoch: 30 - Training Cost: 0.012900685891509056  Testing Cost: 0.014597556553781033
Epoch: 35 - Training Cost: 0.011000080034136772  Testing Cost: 0.012180657126009464
Epoch: 40 - Training Cost: 0.010341642424464226  Testing Cost: 0.01130259782075882
Epoch: 45 - Training Cost: 0.008823253214359283  Testing Cost: 0.009784952737390995
Epoch: 50 - Training Cost: 0.007908898405730724  Testing Cost: 0.008856650441884995
Epoch: 55 - Training Cost: 0.006876008119434118  Testing Cost: 0.0077279591932892

And now we have a second folder. Now let's open up TensorBoard and see how these runs look. Let's open up a terminal window. 

Open terminal and type "tensorboard --logdir=05/logs".

Directory should be where our logs files are stored.

When TensorBoard starts it will give you a URL to open in your browser. Copy and paste that into your web browser. Okay, now click on logging to see our charts and let's click to expand the chart. 

## Custom data Visulization: Add custom visualizations to TensorBoard

TensorBoard allows us to create custom visualizations beyond just line graphs. We can use these visualizations to monitor your machine learning model and what kind of data it's generating. Currently TensorFlow supports these types of visualizations. First, the image visualization allows us to see any array of data as an image. We can create an image visualization by adding a tf.summary.image object to your graph and passing it the array you want to visualize. This is helpful when you're building a neural network that classifies or generates images.

We can also listen to audio data in TensorBoard. To add an audio player to TensorBoard, we can create a new tf.summary.audio object and you add it to your computational graph, this is typically used when building models that recognize speech or generate sounds. It lets us hear the sound files that our model is processing.

We can also create interactive histograms and distribution graphs in TensorBoard. When you add a tf.summary.histogram object to our computational graph, it creates both a histogram like you see here, and a distribution graph. Histograms are a powerful way to monitor ranges of values over time, they show us not only the range of values in the array of data but they can also display how these data ranges vary through time.

![Custom Data Visulization](images/data_viz.png)



Let's try adding a histagram summary to a neural network and see how it works. 

we've already defined our computational graph in the training loop. We also already have a scalar summary defined that monitors the cost of our neural network as it's trained. This will generate a line chart that shows the neural network getting more accurate over time during the training process. But let's say we want to visualize the actual predictions that our neural network is making during the training process. To do that, we can add a histogram that monitors our predictions.

Let's create a new tf.summary.histogram node. Tf.summary.histogram then we'll pass in the name for our node in this case we'll use predicted value. And then we pass in the node in our graph that we want to monitor, which will be prediction. And that's all we have to do. Since we already called tf.summary.merge_all on the next line, any new summary metrics we add to our graph will automatically get picked up and added to our log files. Let's run the code to train our network and generate log files that we can view in TensorBoard. But first, if you already have a log subfolder, delete that before continuing. 

In [12]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
tf.reset_default_graph() 


# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

# Define model parameters
RUN_NAME = "histogram_visualization"
learning_rate = 0.001
training_epochs = 100

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Section One: Define the layers of the neural network itself


# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="X")

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.glorot_uniform_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# Section Two: Define the cost function of the neural network that will be optimized during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

####################################################################################
# Adding Histogram here
####################################################################################
    
# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    tf.summary.histogram('predicted_value', prediction)
    summary = tf.summary.merge_all()

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every few training steps, log our progress
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))


Epoch: 0 - Training Cost: 0.016277538612484932  Testing Cost: 0.01728086546063423
Epoch: 5 - Training Cost: 0.008747064508497715  Testing Cost: 0.009324006736278534
Epoch: 10 - Training Cost: 0.0066581652499735355  Testing Cost: 0.006988320965319872
Epoch: 15 - Training Cost: 0.005339732393622398  Testing Cost: 0.005356414243578911
Epoch: 20 - Training Cost: 0.003938702400773764  Testing Cost: 0.004059947095811367
Epoch: 25 - Training Cost: 0.002515074796974659  Testing Cost: 0.002571344142779708
Epoch: 30 - Training Cost: 0.0016279771225526929  Testing Cost: 0.0018159097526222467
Epoch: 35 - Training Cost: 0.0011463374830782413  Testing Cost: 0.0013736814726144075
Epoch: 40 - Training Cost: 0.0008409644942730665  Testing Cost: 0.0010576178319752216
Epoch: 45 - Training Cost: 0.00064996094442904  Testing Cost: 0.0008440464735031128
Epoch: 50 - Training Cost: 0.0005057525704614818  Testing Cost: 0.0007262466824613512
Epoch: 55 - Training Cost: 0.0003924186748918146  Testing Cost: 0.0005